In [71]:
from functools import reduce
import numpy as np
import pandas as pd
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

# Import, check and prepare data

Import CSV files that contain:
- The PIDs of all datasets published by many repositories that use the Dataverse software and which repositories published them
- Author field metadata entered in all of those datasets

In [2]:
# Get PIDs of all datasets and which repositories published them
datasetPIDsDF = (pd
    .read_csv(
        'dataset_pids_from_most_known_dataverse_installations_2023.08.csv',
        sep=',',
        na_filter=False)
    .query(
        '(dataverse_json_export_saved == True) and\
        (dataverse_installation_name != "ODISSEI_Portal")')
    .drop(columns=['dataverse_json_export_saved'])
    .reset_index(drop=True, inplace=False)
 )

datasetPIDsDF.head()

,dataverse_installation_name,dataset_pid_url,dataverse_collection_alias,dataverse_collection_name,dataverse_collection_type
0,UNB_Libraries_Dataverse,https://doi.org/10.25545/NVN79Z,snb,Service New Brunswick,ORGANIZATIONS_INSTITUTIONS
1,UNB_Libraries_Dataverse,https://doi.org/10.25545/YB60PU,blightchemistry,Blight Chemistry Research Omniverse,RESEARCH_GROUP
2,UNB_Libraries_Dataverse,https://doi.org/10.25545/WHD3KC,blightchemistry,Blight Chemistry Research Omniverse,RESEARCH_GROUP
3,UNB_Libraries_Dataverse,https://doi.org/10.25545/6TZWCG,Ir-CationBinding,Elucidation of Charge Contributions in Iridium-Chelated Hydrogen-Bonding Systems,JOURNALS
4,UNB_Libraries_Dataverse,https://doi.org/10.25545/NYM13B,IFMLAB,Integrated Forest Management Lab,RESEARCH_GROUP


In [18]:
# Get Author field metadata entered in all datasets in Dataverse repositories
# and remove datasets have have no author metadata
authorMetadataDF = (pd
    .read_csv(
        'author(citation)_2023.08.22-2023.08.28.csv',
        sep=',',
        na_filter=True)
    .drop(columns=['dataset_pid', 'authorAffiliation'])
    .query('authorName != "N/A"')
    .reset_index(drop=True, inplace=False)
    )

authorMetadataDF.head()

,dataset_pid_url,dataset_publication_date,dataset_version_number,dataset_version_create_time,authorName,authorIdentifierScheme,authorIdentifier
0,https://doi.org/10.48370/OFD/8TTRLC,2022-05-13,1.0,2022-05-11T14:44:29Z,"Mammal Research Institute, Polish Academy of Sciences",NaN,NaN
1,https://doi.org/10.17026/dans-26a-cq4r,2015-09-18,2.0,2022-02-18T19:29:27Z,H.J. Hesseling,NaN,NaN
2,https://doi.org/10.17026/dans-zc2-mc2g,2016-12-31,1.0,2022-02-24T21:01:03Z,S. Moerman,NaN,NaN
3,https://doi.org/10.17026/dans-x9z-bmn6,2020-12-22,1.0,2022-02-14T05:11:03Z,G. Zielman,NaN,NaN
4,https://doi.org/10.7910/DVN/QD0V0H,2021-09-20,1.0,2021-07-29T19:45:43Z,"Master, Daniel M.",NaN,NaN


In [19]:
print(f'Number of datasets in datasetPIDsDF: {len(datasetPIDsDF)}')
datasetCountInAuthorMetadataDF = len(pd.unique(authorMetadataDF['dataset_pid_url']))
print(f'Number of datasets in authorMetadataDF: {datasetCountInAuthorMetadataDF}')

Number of datasets in datasetPIDsDF: 390401
Number of datasets in authorMetadataDF: 390401


In [28]:
# Join the datasetPIDsDF and the authorMetadataDF to add the installation column,
# so we know which installations published each dataset
# and remove datasets that have no author

datasetPIDsAndAuthorMetadataDF = (pd
    .merge(datasetPIDsDF, authorMetadataDF,
        how='inner',
        on=['dataset_pid_url'])
    .drop(columns=[
        'dataset_version_number',
        'dataset_version_create_time'])
    .reset_index(drop=True, inplace=False))

# Make sure the count of rows in datasetPIDsAndAuthorMetadataDF
# is the same as the count of total datasets in datasetPIDsDF: 390401
datasetCountInDatasetPIDsAndAuthorMetadataDF = len(pd.unique(datasetPIDsAndAuthorMetadataDF['dataset_pid_url']))
print(f'Number of datasets in datasetPIDsAndAuthorMetadataDF: {datasetCountInDatasetPIDsAndAuthorMetadataDF}')

print(f'Number of author metadata in datasetPIDsAndAuthorMetadataDF: {len(datasetPIDsAndAuthorMetadataDF)}')

countOfInstallations = len(datasetPIDsAndAuthorMetadataDF.dataverse_installation_name.unique())
print(f'Number of installations in datasetPIDsAndAuthorMetadataDF: {countOfInstallations}')

Number of datasets in datasetPIDsAndAuthorMetadataDF: 390401
Number of author metadata in datasetPIDsAndAuthorMetadataDF: 667435
Number of installations in datasetPIDsAndAuthorMetadataDF: 84


In [21]:
datasetPIDsAndAuthorMetadataDF.head()

,dataverse_installation_name,dataset_pid_url,dataverse_collection_alias,dataverse_collection_name,dataverse_collection_type,dataset_publication_date,authorName,authorIdentifierScheme,authorIdentifier
0,UNB_Libraries_Dataverse,https://doi.org/10.25545/NVN79Z,snb,Service New Brunswick,ORGANIZATIONS_INSTITUTIONS,2018-05-10,Service New Brunswick,NaN,NaN
1,UNB_Libraries_Dataverse,https://doi.org/10.25545/YB60PU,blightchemistry,Blight Chemistry Research Omniverse,RESEARCH_GROUP,2018-06-20,"Blight, Barry A",ORCID,0000-0003-1166-6206
2,UNB_Libraries_Dataverse,https://doi.org/10.25545/WHD3KC,blightchemistry,Blight Chemistry Research Omniverse,RESEARCH_GROUP,2019-06-23,"Balonova, Barbora",NaN,NaN
3,UNB_Libraries_Dataverse,https://doi.org/10.25545/6TZWCG,Ir-CationBinding,Elucidation of Charge Contributions in Iridium-Chelated Hydrogen-Bonding Systems,JOURNALS,2021-05-18,"Blight, Barry A",NaN,NaN
4,UNB_Libraries_Dataverse,https://doi.org/10.25545/NYM13B,IFMLAB,Integrated Forest Management Lab,RESEARCH_GROUP,2020-06-30,"Chen, Yingbing",NaN,NaN


# Explore data

In [30]:
# Get info about author metadata that includes no values and any values in the identifier type or identifier fields
authorsWithNoIdentifiersDf = (
    datasetPIDsAndAuthorMetadataDF
        .query(
            'authorIdentifierScheme != authorIdentifierScheme and\
            authorIdentifier != authorIdentifier')
        .reset_index(drop=True, inplace=False)
)

authorsWithPartialIdentifiersDf = (
    datasetPIDsAndAuthorMetadataDF
        .query(
            '(authorIdentifierScheme == authorIdentifierScheme and\
             authorIdentifier != authorIdentifier) or\
            (authorIdentifierScheme != authorIdentifierScheme and\
             authorIdentifier == authorIdentifier)')
        .reset_index(drop=True, inplace=False)
)

authorsWithCompleteIdentifiersDf = (
    datasetPIDsAndAuthorMetadataDF
        .query(
            'authorIdentifierScheme == authorIdentifierScheme and\
             authorIdentifier == authorIdentifier')
        .reset_index(drop=True, inplace=False)
)

sumOfAuthorMetadata = (
    len(authorsWithPartialIdentifiersDf)
    + len(authorsWithCompleteIdentifiersDf)
    + len(authorsWithNoIdentifiersDf)
)
print(f'Number of author metadata with partial, complete, and no identifier metadata: {sumOfAuthorMetadata}')

print(f'Number of author metadata with partial identifier metadata: {len(authorsWithPartialIdentifiersDf)}')
print(f'Number of author metadata with complete identifier metadata: {len(authorsWithCompleteIdentifiersDf)}')
print(f'Number of author metadata with no identifier metadata: {len(authorsWithNoIdentifiersDf)}')

Number of author metadata with partial, complete, and no identifier metadata: 667435
Number of author metadata with partial identifier metadata: 4438
Number of author metadata with complete identifier metadata: 82078
Number of author metadata with no identifier metadata: 580919


Get count of metadata for each installation

In [40]:
# Count of author metadata in authorsWithNoIdentifiersDf per installation
authorsWithNoIdentifiersByInstallationDf = (
    authorsWithNoIdentifiersDf
        .value_counts(subset=['dataverse_installation_name'])
        .to_frame('count_of_authors_with_no_identifiers')
)

authorsWithNoIdentifiersByInstallationDf.head()

,count_of_authors_with_no_identifiers
dataverse_installation_name,
DANS_Data_Station_Archaeology,149262
Harvard_Dataverse,143643
JPL_Open_Repository,134454
Open_Forest_Data,65361
Borealis,28295


In [41]:
# Count of author metadata in authorsWithPartialIdentifiersDf per installation
authorsWithPartialIdentifiersByInstallationDf = (
    authorsWithPartialIdentifiersDf
        .value_counts(subset=['dataverse_installation_name'])
        .to_frame('count_of_authors_with_partial_identifiers')
)

authorsWithPartialIdentifiersByInstallationDf.head()

,count_of_authors_with_partial_identifiers
dataverse_installation_name,
Harvard_Dataverse,1701
International_Potato_Center,918
Edmond,766
CIMMYT_Research_Data,155
Borealis,122


In [42]:
# Count of author metadata in authorsWithCompleteIdentifiersDf per installation
authorsWithCompleteIdentifiersByInstallationDf = (
    authorsWithCompleteIdentifiersDf
        .value_counts(subset=['dataverse_installation_name'])
        .to_frame('count_of_authors_with_complete_identifiers')
)

authorsWithCompleteIdentifiersByInstallationDf.head()

,count_of_authors_with_complete_identifiers
dataverse_installation_name,
Harvard_Dataverse,23127
Open_Forest_Data,14340
DaRUS,6991
DataverseNL,6972
Borealis,3707


In [58]:
# Join authorsWithNoIdentifiersByInstallationDf, authorsWithPartialIdentifiersByInstallationDf, and authorsWithCompleteIdentifiersByInstallationDf
dataframes = [
    authorsWithNoIdentifiersByInstallationDf,
    authorsWithPartialIdentifiersByInstallationDf,
    authorsWithCompleteIdentifiersByInstallationDf
]

countOfAuthorsByInstallationDf = reduce(lambda left, right: left.join(right, how='outer'), dataframes)
countOfAuthorsByInstallationDf = (
    countOfAuthorsByInstallationDf
        .fillna(0)
        .astype('int32')
        .reset_index(drop=False, inplace=False)
        # Reorder columns
        .loc[:,[
            'dataverse_installation_name',
            'count_of_authors_with_complete_identifiers',
            'count_of_authors_with_partial_identifiers',
            'count_of_authors_with_no_identifiers']]
)

countOfAuthorsByInstallationDf.head()

,dataverse_installation_name,count_of_authors_with_complete_identifiers,count_of_authors_with_partial_identifiers,count_of_authors_with_no_identifiers
0,ACSS_Dataverse,5,0,141
1,ADA_Dataverse,1102,22,2140
2,ASU_Library_Research_Data_Repository,92,4,115
3,AUSSDA_Dataverse,258,0,2104
4,Abacus,0,1,4083


In [67]:
# Get number of author metadata with each type of identifier, e.g. ORCID, GND
# authorsWithCompleteIdentifiersDf

identifierTypesByInstallationDf = (pd
    .crosstab(
        authorsWithCompleteIdentifiersDf.dataverse_installation_name,
        authorsWithCompleteIdentifiersDf.authorIdentifierScheme,
        margins=True, margins_name='Total')

    # .reset_index(drop=False, inplace=False)
)

identifierTypesByInstallationDf.head()


authorIdentifierScheme,DAI,DOI,GND,ISNI,LCNA,ORCID,ResearcherID,ScopusID,VIAF,idHAL,Total
dataverse_installation_name,,,,,,,,,,,
ACSS_Dataverse,0,0,0,0,0,5,0,0,0,0,5
ADA_Dataverse,0,0,0,0,0,1102,0,0,0,0,1102
ASU_Library_Research_Data_Repository,0,0,0,0,0,92,0,0,0,0,92
AUSSDA_Dataverse,0,0,0,0,0,258,0,0,0,0,258
Arca_Dados,0,0,0,0,0,132,0,0,0,0,132


In [ ]:


# Number of author metadata values with ORCIDs
authorMetadataWithORCIDs = (
    authorMetadataWithIdentifiers
        .query(
        '~authorIdentifier.str.contains("orcid.org").values or\
        authorIdentifierScheme == "ORCID"')
        .reset_index(drop=True, inplace=False)
)

authorMetadataWithORCIDs.head()